# Solución al problema planteado proyecto de ejemplo
## Se importan todas las librerías a usar

### Librería pandas para usar tablas

In [1]:
import pandas as pd

### Librería requests para consultar la api

In [2]:
import requests

### librería Cartesian de sklearn para crear todas las posibles combinaciones

In [3]:
from sklearn.utils.extmath import cartesian

## Inicio de la descarga de datos

In [4]:
uri = 'https://mach-eight.uc.r.appspot.com/'
json_data = requests.get(uri).json()
df = pd.json_normalize(json_data['values'])

In [5]:
df['h_in']=df['h_in'].astype(int)
df.reset_index(drop=True, inplace=True)
df.head()

,first_name,h_in,h_meters,last_name
0,Alex,77,1.96,Acker
1,Hassan,76,1.93,Adams
2,Arron,77,1.96,Afflalo
3,Maurice,77,1.96,Ager
4,Alexis,84,2.13,Ajinca


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  435 non-null    object
 1   h_in        435 non-null    int64 
 2   h_meters    435 non-null    object
 3   last_name   435 non-null    object
dtypes: int64(1), object(3)
memory usage: 13.7+ KB


### Procedimiento que calcula las sumas de alturas

In [7]:
def calcular_suma(df, target, partial=[], partial_sum=0):
    """
    Calcula de forma recursiva la suma de columnas en un dataframe
    """
    if partial_sum == target:
        yield partial
    if partial_sum > target:
        return
    for i, n in enumerate(df['h_in']):
        remaining = df.loc[i + 1:]
        yield from calcular_suma(remaining, target, partial + [n], partial_sum + n)

### Establecer la suma de alturas que se desea consultar

In [8]:
altura_pulgadas = 139

In [9]:
df_unico = pd.DataFrame(df['h_in'].drop_duplicates()).reset_index(drop=True)
alturas = list(calcular_suma(df_unico, altura_pulgadas))

In [10]:
indices = cartesian((df[df['h_in'].isin(alturas[0])].index, df[df['h_in'].isin(alturas[0])].index))

In [11]:
izquierda = df.loc[indices[:,0]].reset_index()
derecha = df.loc[indices[:,1]].reset_index()
combinados = pd.concat([izquierda, derecha], axis=1)
combinados.columns = ['index_l', 'first_name_l', 'h_in_l', 'h_meters_l', 'last_name_l', 'index_r', 'first_name_r', 'h_in_r', 'h_meters_r', 'last_name_r']
combinados['sum'] = combinados['h_in_l'] + combinados['h_in_r']
combinados['nuevo_index'] = combinados['index_l'] + combinados['index_r']
combinados = combinados[combinados['sum'] == altura_pulgadas]
combinados.drop_duplicates(subset='nuevo_index', keep='first', inplace=True)
combinados.drop(columns=['index_l', 'index_r', 'nuevo_index', 'h_meters_l', 'h_meters_r'], inplace=True)

### Resultado de la combinación

In [12]:
combinados

,first_name_l,h_in_l,last_name_l,first_name_r,h_in_r,last_name_r,sum
1,Brevin,70,Knight,Nate,69,Robinson,139
5,Nate,69,Robinson,Mike,70,Wilks,139
